## General Colab Tips
- Modify files by opening/editing them in the UI (double-click to open).
- `Right click > Refresh` in the Colab file explorer to update the directory.
- All files are lost when the Colab session disconnects, so make sure back up your work.
- Do **not** use `drive.mount` for your datasets! Reading from GDrive is super slow.
- Instead, place datasets into the `/content/` folder and modify your data accordingly.

**Make a copy of this notebook and modify this to whatever workflow you prefer!**

If you have some additional colab tips, please share them on the discussion forum.

## Setup

First, enable a GPU runtime via `Runtime > Change runtime type > T4 GPU`

Next, upload the your project files to the Colab. You can do this by either
- using Github (**recommended**)
- uploading files manually using the UI

## Github Setup

You can use git from within Google Colab!

For this section, we assume you know how to use git and have already pushed the starter code to a private repo.

Before you continue, make sure you download and push the starter code to your repo.  
It's a good idea to structure your repo something like
```
online_deep_learning/
    homework1/
    homework2/
    ...
```

We highly recommend using this workflow as you'll be able to easily pull/commit your changes after modifying your model on Colab.

To do this, you'll need a personal access token from [https://github.com/settings/tokens](https://github.com/settings/tokens)

The easiest thing to do is select "classic" token and make sure you have the `repo` scope selected to allow access to your private repos.
There's also fine-grained tokens where you can select access to specific repos.

Once you have your token, fill in your information and then run the following cell to clone your git repo to the Colab instance.

In [43]:
import os


os.environ['USER'] = 'thisisanu'
os.environ['REPO'] = 'online_deep_learning'
os.environ['TOKEN'] = 'ithub_pat_11BIG54WA0sAoSEImPXImB_OUcAjo84vwWTugnGOH4GbPnqToNKRJESKhyxDDzjYYUDQXP7HGCBmOPHCVt'

# do everything in colab's "root" directory
!rm -rf /content/online_deep_learning
!rm -rf /content/online_deep_learning/homework4

%cd /content
!git clone https://${TOKEN}@github.com/${USER}/${REPO}.git

# make sure your repo shows up
%ls

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
/content
Cloning into 'online_deep_learning'...
remote: Enumerating objects: 1493, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 1493 (delta 145), reused 1 (delta 1), pack-reused 1301 (from 5)
Receiving objects: 100% (1493/1493), 3.87 MiB | 8.00 MiB/s, done.
Resolving deltas: 100% (1054/1054), done.
online_deep_learning/  sample_data/


## Code Setup

Move into `homework4/` so we can continue setting up the data / code for training.

This will be the main working directory and the training/grading must be run from this directory.


In [44]:
# navigate to your repo
%cd /content/{os.environ['REPO']}
%ls

# go to a specific homework
%cd homework4
%ls


/content/online_deep_learning
gitignore  homework1/  homework2/  homework3/  homework4/  README.md
/content/online_deep_learning/homework4
assets/    grader/    Homework_4.ipynb  requirements.txt
bundle.py  homework/  README.md


## Dataset Setup

Now that your code is all ready, the next step is to download the datasets.

Note: it's good practice to add data directories like `*/drive_data` to your `.gitignore` so you don't accidently commit them to your repo.

Since the datasets used in this class are relatively small, we can simply re-download them if the compute instance crashes/restarts.

In [45]:
!rm -rf drive_data
!curl -s -L https://www.cs.utexas.edu/~bzhou/dl_class/drive_data.zip -o ./drive_data.zip && unzip -qo drive_data.zip
%ls

import sys
import torch

!{sys.executable} -m pip install -r requirements.txt
!{sys.executable} -m pip install --upgrade "ipython>=8.20.0"

print("Python:", sys.version)
print("Executable:", sys.executable)

!pip show ipython

assets/    drive_data/     grader/    Homework_4.ipynb  requirements.txt
bundle.py  drive_data.zip  homework/  README.md
Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Executable: /usr/bin/python3
Name: ipython
Version: 9.8.0
Summary: IPython: Productive Interactive Computing
Home-page: https://ipython.org
Author: The IPython Development Team
Author-email: ipython-dev@python.org
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: decorator, ipython-pygments-lexers, jedi, matplotlib-inline, pexpect, prompt_toolkit, pygments, stack_data, traitlets
Required-by: bigquery-magics, cufflinks, google-colab, ipykernel, ipyparallel, ipython-sql, ipywidgets, jupyter-console, spanner-graph-notebook


## Model Implementation + Training

Now you should be all set up.
Next, you'll need to implement
- `homework/train_planner.py`
- `homework/models.py`

And then you're ready to train

In [ ]:
# ---------------------------------------------------------------
# Full MLPPlanner training snippet
# ---------------------------------------------------------------

%reload_ext autoreload
%autoreload 2

import torch
from homework.train_planner import train

# Use GPU if available, else CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# Training configuration (common settings)
train_config = {
    "transform_pipeline": "state_only",  # for MLP and Transformer
    "num_workers": 2,
    "lr": 3e-4,
    "batch_size": 128,
    "num_epoch": 40,
    "device": device,
}

# List of planners to train
planners = ["mlp_planner", "transformer_planner", "cnn_planner"]

for planner_name in planners:
    print(f"\n=== Training {planner_name} ===\n")

    # Update the model name in config
    train_config["model_name"] = planner_name

    # For CNNPlanner, use "image_only" transform
    train_config["transform_pipeline"] = "state_only"

    # Start training
    train(**train_config)




=== Training mlp_planner ===

Using device: cpu
Model: mlp_planner
Loaded 8000 samples from 16 episodes
Loaded 2000 samples from 4 episodes
Using weighted MSE loss for better lateral error performance
Using AdamW optimizer, lr=0.0003, weight_decay=0.001
Starting training...
Epoch 1/50: Train Loss: 66.2435, Val Long Error: 0.2225, Val Lat Error: 3.0856, LR: 0.000300
New best lateral error: 3.0856
New best longitudinal error: 0.2225
Saved model → /content/online_deep_learning/homework4/homework/mlp_planner.th
Saved best model to /content/online_deep_learning/homework4/homework/mlp_planner.th
Epoch 2/50: Train Loss: 27.6464, Val Long Error: 0.2336, Val Lat Error: 0.9186, LR: 0.000300
New best lateral error: 0.9186
Saved model → /content/online_deep_learning/homework4/homework/mlp_planner.th
Saved best model to /content/online_deep_learning/homework4/homework/mlp_planner.th
Epoch 3/50: Train Loss: 9.8264, Val Long Error: 0.1672, Val Lat Error: 0.4796, LR: 0.000300
New best lateral error: 

## Grader

Run the following cell to grade your homework.

Note: if you don't set up PySuperTuxKart, the grader will not run the driving tests.

In [ ]:
!cd /content/online_deep_learning/homework4/homework
!python3 -m grader homework -vv --disable_color


## PySuperTuxKart Setup (Optional)

We will use your trained planner to drive around in SuperTuxKart!  
SuperTuxKart is a python wrapper around a C++ game, so it requires a few more build steps.

This is optional to test locally - if your planner passes the local grader's tests for prediction accuracy, it should drive just fine and you can submit to the online grader. You only need to set up PySuperTuxKart locally if you want to see your model driving around.

In [ ]:
# don't worry about the "... is not a symbolic link" logs
!sudo DEBIAN_FRONTEND=noninteractive apt install -qq libnvidia-gl-535
!pip install PySuperTuxKartData --index-url=https://www.cs.utexas.edu/~bzhou/dl_class/pystk
!pip install PySuperTuxKart --index-url=https://www.cs.utexas.edu/~bzhou/dl_class/pystk

The following additional packages will be installed:
  libnvidia-common-535
The following NEW packages will be installed:
  libnvidia-common-535 libnvidia-gl-535
0 upgraded, 2 newly installed, 0 to remove and 41 not upgraded.
Need to get 183 MB of archives.
After this operation, 464 MB of additional disk space will be used.
Selecting previously unselected package libnvidia-common-535.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../libnvidia-common-535_535.274.02-0ubuntu1_all.deb ...
Unpacking libnvidia-common-535 (535.274.02-0ubuntu1) ...
Selecting previously unselected package libnvidia-gl-535:amd64.
Preparing to unpack .../libnvidia-gl-535_535.274.02-0ubuntu1_amd64.deb ...
dpkg-query: no packages found matching libnvidia-gl-450
Unpacking libnvidia-gl-535:amd64 (535.274.02-0ubuntu1) ...
Setting up libnvidia-common-535 (535.274.02-0ubuntu1) ...
Setting up libnvidia-gl-535:amd64 (535.274.02-0ubuntu1) ...
Processing triggers for libc-bin 

## End to End Driving Visualization

After your models are trained, you can see how they perform in game!

Some of the driving might look "jittery", due to the simple logic of the controller (directly steer towards the predicted waypoints).

You can always tweak the controller for fun, but note that we will use the original one for grading.

In [ ]:
# This script will simply run your model and save the session as a video
!python3 -m homework.supertux_utils.evaluate --model mlp_planner --track lighthouse --max-steps 100

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/online_deep_learning/homework4/homework/supertux_utils/evaluate.py", line 297, in <module>
    main()
  File "/content/online_deep_learning/homework4/homework/supertux_utils/evaluate.py", line 278, in main
    model = load_model(args.model, with_weights=True)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/online_deep_learning/homework4/homework/models.py", line 137, in load_model
    assert model_path.exists(), f"{model_path.name} not found"
           ^^^^^^^^^^^^^^^^^^^
AssertionError: mlp_planner.th not found


In [ ]:
# This cell displays the videos
from IPython.display import Video
from pathlib import Path


video_dir = Path("videos")

for video_path in sorted(video_dir.glob("*.mp4")):
    print(video_path)
    display(Video(video_path, embed=True))

## Update your changes


In [ ]:
%ls
!git status

# Be careful not to "git add *" since there are datasets and logs
!git add homework/*.py
!git config --global user.email "GITHUB_EMAIL"
!git config --global user.name "GITHUB_USER"
!git commit -m "update"
!git push origin main

## Submission

Run the following cell to bundle your submission (modify UTID accordingly).

If you notice that your bundle is too large, you can modify the `bundle.py` script and ignore large files by adding them manually to `BLACKLIST`.

After the bundler and grader run, right click and download your bundled `.zip` file from the Colab UI.


In [ ]:
!python3 bundle.py homework UTID

# optional: run the grader with your bundled homework to double check
!python3 -m grader UTID.zip -vv --disable_color

## Tensorboard (Optional)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
!sudo DEBIAN_FRONTEND=noninteractive apt install -qq libnvidia-gl-535
!pip install PySuperTuxKartData --index-url=https://www.cs.utexas.edu/~bzhou/dl_class/pystk -vv
!pip install PySuperTuxKart --index-url=https://www.cs.utexas.edu/~bzhou/dl_class/pystk -vv

In [ ]:
!python -v